In [1]:
import camelot
import tabula
import pandas as pd
import numpy as np

Hier lese ich alle Seiten des PDF mit dem Flavor "Stream aus, weil der besser funktioniert hat:

In [2]:
pfad = "Lieferung_SEB-Schulergebnisse_2/Schulberichte/AlleSchulen_2006.pdf"
umfrage = camelot.read_pdf(pfad, flavor='stream', pages='all')

In [3]:
umfrage

<TableList n=42>

Wir schauen uns mal eine einzelne Tabelle an (relevant sind alle mit geraden Zahlen: 0,2,4 etc.)

In [4]:
df = umfrage[0].df

In [5]:
df.head(21)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,,,,,,Anteil der positiven,,,,,,,,,,,,,,
1,,,,,,,,,,,,,,Beurteilung,,,,,,
2,Büelrain,,,,,Bewertungen (%),,,,,,,,,,,,,,
3,,,,,diese,,,,,negativ ←→ positiv,,,,,,negativ ←→ positiv,,,,
4,,,Anzahl,Antworten,Gruppe,alle,Abweichung,,,,absolute Anzahl,,,,,,Prozent,,,
5,Frage (Nr.),,,,2006 2006,,zu allen,,1,2,3,4,5,6,1,2,3,4,5,6
6,(6),Zufriedenheit mit der Mittelschule,,65,87.7,90.2,-2.5,,0,1,7,12,28,17,0,2,11,18,43,26
7,,in fachlicher Hinsicht,,62,75.8,74.0,+1.8,,0,6,9,20,22,5,0,10,15,32,35,8
8,Vorbereitung,,,,,,,,,,,,,,,,,,,
9,,in persönlichkeitsbildender Hinsicht,,62,77.4,74.9,+2.5,,2,5,7,25,19,4,3,8,11,40,31,6


In [6]:
df.shape

(83, 20)

df.columns.values

#### Finden der relevanten Felder
1. Hier holen wir den Schulnamen her:

In [7]:
Schulname = df.loc[2,0]
Schulname

'Büelrain'

2. Hier sind die Noten zum ersten Wert (Zufriedenheit), von denen wir für im 2006er-File manuell den Schnitt generieren müssen:

In [8]:
Zufriedenheit1 = df.loc[6,8] # Note 1 ist loc[6,8], Note 6 ist loc[6,13]
Zufriedenheit1

'0'

In [9]:
#Die Anzahl Antworten finden wir hier:
Antworten = df.loc[6,3]

Dummerweise merken wir jetzt, dass die entsprechenden Zahlen alle als Strings erfasst sind.

In [10]:
df.dtypes

0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
dtype: object

 Deshalb werden wir:
1. Den Kopf wegschneiden, der sowiso keine Zahlen enthält.
(**ACHTUNG** Diesen Schritt dürfen wir erst machen, nachdem wir den Schulnamen rausgelsen haben, der fehlt sonst nachher. **Noch besser**: Wir generieren ein neues kurz-Datarame, mit dem wir dann arbeiten.
2. wir müssen die für uns wichtigen Spalten 3 und 8 bis 13 in floats umbauen:

In [11]:
#Schitt 1:
df_kurz = df.drop([0,1,2,3,4,5])
df_kurz.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
6,(6),Zufriedenheit mit der Mittelschule,,65,87.7,90.2,-2.5,,0,1,7,12,28,17,0,2,11,18,43,26
7,,in fachlicher Hinsicht,,62,75.8,74.0,+1.8,,0,6,9,20,22,5,0,10,15,32,35,8
8,Vorbereitung,,,,,,,,,,,,,,,,,,,


In [12]:
df_kurz[3].value_counts()

65    22
0     15
       4
1      4
56     4
59     3
62     3
57     3
3      2
2      2
42     2
11     2
63     1
15     1
54     1
9      1
12     1
14     1
64     1
4      1
5      1
51     1
30     1
Name: 3, dtype: int64

In [13]:
#Das Problem sind offensichtlich die Leerschläge, deshalb ersetzen wir diese mit NaN (dafür numpy importieren!)
df_kurz = df_kurz.replace("", np.nan)

In [14]:
# Jetzt können wir die Spalten in Floats umwandeln und prüfen die Typen:
Spaltenliste = [3,8,9,10,11,12,13]
df_kurz[Spaltenliste] = df_kurz[Spaltenliste].astype(float)
df_kurz.dtypes

0      object
1      object
2      object
3     float64
4      object
5      object
6      object
7      object
8     float64
9     float64
10    float64
11    float64
12    float64
13    float64
14     object
15     object
16     object
17     object
18     object
19     object
dtype: object

Jetzt suchen wir die Zeilen, in denen im df_kurz die uns interssierenden Werte versteckt sind.
(Von denen wir dann die Notenschnitte berechnen wollen)
Es sind die Zeilen:
6 = Zufriedenheit
7 = Vorbereitung_Fach
9 = Vorbereitung_Pers
13 = Problemloesung
14 = Urteilskraft
16 = Gruppenarbeit
17 = Konfliktfähigkeit

In [15]:
#hier suchen wir gezielt nach dem Ausschnitt, deruns interessiert:
x= range(7,20) #Die range der Zeilen, die wir anschauen wollen
df_kurz.loc[x]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
7,NaN,in fachlicher Hinsicht,NaN,62.0,75.8,74.0,+1.8,NaN,0.0,6.0,9.0,20.0,22.0,5.0,0,10,15,32,35,8
8,Vorbereitung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,in persönlichkeitsbildender Hinsicht,NaN,62.0,77.4,74.9,+2.5,NaN,2.0,5.0,7.0,25.0,19.0,4.0,3,8,11,40,31,6
10,Studium (5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,insgesamt,NaN,62.0,79.0,80.5,-1.4,NaN,0.0,3.0,10.0,20.0,26.0,3.0,0,5,16,32,42,5
12,NaN,neues Wissen zu erwerben.,NaN,65.0,98.5,96.4,+2.0,NaN,0.0,0.0,1.0,7.0,28.0,29.0,0,0,2,11,43,45
13,NaN,selbständig Probleme zu lösen.,NaN,65.0,78.5,83.0,-4.5,NaN,0.0,6.0,8.0,18.0,28.0,5.0,0,9,12,28,43,8
14,NaN,eigenständig zu urteilen.,NaN,65.0,70.8,82.5,-11.7,–,2.0,6.0,11.0,14.0,22.0,10.0,3,9,17,22,34,15
15,Persönliche \nEntwicklung (1)\ngelernt . . .,zielorientiert zu handeln.,NaN,65.0,72.3,80.6,-8.3,NaN,2.0,8.0,8.0,17.0,21.0,9.0,3,12,12,26,32,14
16,NaN,in Gruppen zu arbeiten.,NaN,65.0,76.9,76.2,+0.7,NaN,1.0,2.0,12.0,23.0,25.0,2.0,2,3,18,35,38,3


In [16]:
#Nun testen wir mal, ob wir für einen Wert den den Schnitt berechnen
Notensumme = (df_kurz.loc[6,8]*1) + (df_kurz.loc[6,9]*2) + (df_kurz.loc[6,10]*3) + (df_kurz.loc[6,11]*4) + (df_kurz.loc[6,12]*5) + (df_kurz.loc[6,13]*6)
Antworten = df_kurz.loc[6,3]
Notenschnitt = Notensumme / Antworten
Notenschnitt
# Bingo!

4.815384615384615

Jetzt sollten wir die Schnitte für all jene Werte berechnen können, die uns interessieren, und diese jeweils in einer neuen Spalte im Ursprungs-Dataframe(df) abspeichern:

Wir vereinfachen das ganze etwas, indem wir eins Spalte Summe machen, in der alle multiplizierten werte summiert sind. Also:

In [17]:
#Die Spalten mit den Noten 1 bis 6 sind wie gesagt die Spalten 8 bis 13:
df_kurz["Summe"] = df_kurz[8] + df_kurz[9].multiply(2) + df_kurz[10].multiply(3) + df_kurz[11].multiply(4) + df_kurz[12].multiply(5) + df_kurz[13].multiply(6)

# und diesen Wert teilen wir nun durch die Anzahl antworten (Column 3) und machen daraus den Notenschnitt:

df_kurz["Notenschnitt"] = (df_kurz["Summe"] / df_kurz[3]).round(1) #gerundet auf eine Kommastelle

In [18]:
df_kurz

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,Summe,Notenschnitt
6,(6),Zufriedenheit mit der Mittelschule,NaN,65.0,87.7,90.2,-2.5,NaN,0.0,1.0,...,28.0,17.0,0,2,11,18,43,26,313.0,4.8
7,NaN,in fachlicher Hinsicht,NaN,62.0,75.8,74.0,+1.8,NaN,0.0,6.0,...,22.0,5.0,0,10,15,32,35,8,259.0,4.2
8,Vorbereitung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,in persönlichkeitsbildender Hinsicht,NaN,62.0,77.4,74.9,+2.5,NaN,2.0,5.0,...,19.0,4.0,3,8,11,40,31,6,252.0,4.1
10,Studium (5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,NaN,* wurde bei Problemen von der Schule oft allei...,NaN,65.0,89.2,78.4,+10.9,+,0.0,0.0,...,33.0,7.0,0,0,11,28,51,11,300.0,4.6
79,NaN,* Differenzen mit Lehrpersonen belasteten mich...,NaN,64.0,81.3,79.5,+1.8,NaN,0.0,2.0,...,24.0,15.0,0,3,16,20,38,23,296.0,4.6
80,NaN,* Absenzen waren ein grosses Konfliktthema,NaN,65.0,73.8,81.7,-7.9,NaN,4.0,7.0,...,15.0,24.0,6,11,9,14,23,37,291.0,4.5
81,NaN,NaN,"* Frage negativ gestellt, Skala gekehrt: posi...",NaN,NaN,NaN,NaN,Vorzeichen bedeutet signifikante Abweichung di...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Was  haben wir bis jetzt?
Wir haben jetzt alle gesuchten Daten für eine Schule
- diese können wir nun direkt ansteuern und uns aus den Werten ein Mini-Dict zusammensetzen
- Dieses wiederum können wir in einen Loop einfügen und daraus eine Liste machen
- diese bauen wir dan zu unserem neuen Dataframe 2006 zusammen
- und dieses exportieren wir zu guter letzt als CSV-Doc

Das Problem unsereres Loops: Wir wollen ja nur jeden zweite Table im Camelot-Import (umfrage) ansteuern. Das geht evtl so:

for i in range(0, len(a), 2): 

In [19]:
for table in range(0, len(umfrage), 2):
    df_schule = umfrage[table].df
    schulname = df_schule.loc[2,0]
    print(schulname)

Büelrain
Enge
Freudenberg
Glattal
Hohe Promenade

Im Lee


Liceo Artistico
Limmattal
Oerlikon
Rämibühl LG
Rämibühl MNG

Rychenberg



Zürcher Oberland
Zürcher Unterland


Ooops, manche der 21 Schulen haben wirnicht erwischt! Was ist a los? betrifft zum Beispiel Tabele Nummer 10 > prüfen:

In [20]:
df_test = umfrage[10].df

In [21]:
df_test.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,,,,,,,,,,,,,,Beurteilung,,,,,,
1,Hottingen,,,,,Bewertungen (%),,,,,,,,,,,,,,
2,,,,,diese,,,,,negativ ←→ positiv,,,,,,negativ ←→ positiv,,,,
3,,,Anzahl,Antworten,Gruppe,alle,Abweichung,,,,absolute Anzahl,,,,,,Prozent,,,
4,Frage (Nr.),,,,2006 2006,,zu allen,,1,2,3,4,5,6,1,2,3,4,5,6


**Erkenntnis:** Bei diesen Tabellen fehl eine Zeile, die müssen wir also in unserem Loop überall dort einbauen

In [22]:
df_newrow = df_test.head(1)   #hier machen wir ein einzeilen-mini-df, dass wir dort jeweils oben an-concatten
df_test2 = pd.concat([df_newrow, df_test], ignore_index = True)   # ignore Index ist wichtig, sons kopiert er den mit und es nützt alls nichts
df_test2.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,,,,,,,,,,,,,,Beurteilung,,,,,,
1,,,,,,,,,,,,,,Beurteilung,,,,,,
2,Hottingen,,,,,Bewertungen (%),,,,,,,,,,,,,,


Schauen, wir, **ob das unser Problem löst**, wenn wir es in den Loop einbauen:

In [23]:
for table in range(0, len(umfrage), 2):
    df_schule = umfrage[table].df
    
    if df_schule.loc[0,13] == "Beurteilung":   #Wir checken hier anhand eines eindeutigen Merkmals, ob eine Zeile fehlt
        df_newrow = df_schule.head(1)
        df_schule = pd.concat([df_newrow, df_schule], ignore_index = True)   # Ignore Index ist entscheiden!              
    schulname = df_schule.loc[2,0]
    print(schulname)

Büelrain
Enge
Freudenberg
Glattal
Hohe Promenade
Hottingen
Im Lee
KME
Küsnacht
Liceo Artistico
Limmattal
Oerlikon
Rämibühl LG
Rämibühl MNG
Rämibühl RG
Rychenberg
Stadelhofen
Wiedikon
Zürich Birch
Zürcher Oberland
Zürcher Unterland


### Yes!
Da hätten wir also unsere erste Liste, jetzt müssen wir das nur noch mit allem zusammenfügen und druchspielen, was wir bisher machten.
Also nochmals von vorne:

In [24]:
df_kurz[3]

6     65.0
7     62.0
8      NaN
9     62.0
10     NaN
      ... 
78    65.0
79    64.0
80    65.0
81     NaN
82     NaN
Name: 3, Length: 77, dtype: float64

In [25]:
jahr = 2006

dic_list = []


for table in range(0, len(umfrage), 2):
    df_schule = umfrage[table].df
    
    if df_schule.loc[0,13] == "Beurteilung":   #Wir checken hier anhand eines eindeutigen Merkmals, ob eine Zeile fehlt
        df_newrow = df_schule.head(1)
        df_schule = pd.concat([df_newrow, df_schule], ignore_index = True)   # Ignore Index ist entscheidend!
        
    name = df_schule.loc[2,0]
    
    df_kurz = df_schule.drop([0,1,2,3,4,5])
    df_kurz = df_kurz.replace("", np.nan)
    our_columns = [3,8,9,10,11,12,13]
    df_kurz[our_columns] = df_kurz[our_columns].astype(float)
    df_kurz["Summe"] = df_kurz[8] + df_kurz[9].multiply(2) + df_kurz[10].multiply(3) + df_kurz[11].multiply(4) + df_kurz[12].multiply(5) + df_kurz[13].multiply(6)
    df_kurz["Notenschnitt"] = (df_kurz["Summe"] / df_kurz[3]).round(3) #gerundet auf eine Kommastelle
    
    zufr = df_kurz.loc[6, "Notenschnitt"]
    vorb_f = df_kurz.loc[7, "Notenschnitt"]
    vorb_p = df_kurz.loc[9, "Notenschnitt"]
    prob = df_kurz.loc[13, "Notenschnitt"]
    urt = df_kurz.loc[14, "Notenschnitt"]
    zus = df_kurz.loc[16, "Notenschnitt"]
    konf = df_kurz.loc[17, "Notenschnitt"]

    mini_dict = {"jahr": jahr, 
                 "name": name, 
                 "zufr": zufr, 
                 "vorb_f": vorb_f, 
                 "vorb_p":vorb_p, 
                 "prob": prob, 
                 "urt": urt, 
                 "zus": zus, 
                 "konf": konf}
    dic_list.append(mini_dict)

In [26]:
new_df = pd.DataFrame(dic_list)
#new_df.sort_values(by=["zufr"], ascending = False)
new_df

,jahr,name,zufr,vorb_f,vorb_p,prob,urt,zus,konf
0,2006,Büelrain,4.815,4.177,4.065,4.277,4.200,4.154,3.738
1,2006,Enge,4.746,4.074,3.926,4.493,4.437,3.986,3.761
2,2006,Freudenberg,4.316,4.000,3.868,4.526,4.658,4.079,3.842
3,2006,Glattal,4.833,3.941,4.118,4.500,4.333,3.722,3.444
4,2006,Hohe Promenade,5.091,4.758,4.636,4.706,4.706,4.000,4.000
5,2006,Hottingen,4.051,3.500,3.711,3.974,4.000,3.821,3.513
6,2006,Im Lee,4.676,4.406,3.971,4.603,4.473,3.836,3.493
7,2006,KME,5.037,4.173,4.257,4.309,4.185,3.963,3.407
8,2006,Küsnacht,5.095,4.119,4.524,4.738,5.095,4.000,4.048
9,2006,Liceo Artistico,5.450,4.579,4.895,4.950,4.950,4.950,4.263


### Achtung: Bei KZO hat es Wert über 6 drin, irgendwas stimmt mit den Werten nicht da nicht (Zahl der Antworten?)

- andere stimmen (manuell überprüft (Enge, Freudenberg, Stadelhofen)
- Erkenntnis: Es ist alles um eine Spalte nach links verschoben: wir müssenunsere Loop also auch noch hier noch um eine Kontrollfunktion erweitern: wenn"Beurteilung im loc[1,12] ist statt auf 13, dann müssen wir die Kolumnen umbenennen

In [34]:
# hier testen wir das an einem einzelnen Problemfall:

df_test = umfrage[38].df

df_test_kurz = df_test.drop([0,1,2,3,4,5,81,82])  # auch Schluss kürzen, weils da Müll drinhat
df_test_kurz = df_test_kurz.replace("", np.nan)
df_test_kurz.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19] # Kolumnenamen umschreiben

In [35]:
our_columns = [3,8,9,10,11,12,13]
df_test_kurz[our_columns] = df_test_kurz[our_columns].astype(float)
df_test_kurz["Summe"] = df_test_kurz[8] + df_test_kurz[9].multiply(2) + df_test_kurz[10].multiply(3) + df_test_kurz[11].multiply(4) + df_test_kurz[12].multiply(5) + df_test_kurz[13].multiply(6)
df_test_kurz["Notenschnitt"] = (df_test_kurz["Summe"] / df_test_kurz[3]).round(3) #gerundet auf eine Kommastelle

df_test_kurz.head(6)

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,Summe,Notenschnitt
6,(6),Zufriedenheit mit der Mittelschule,121.0,90.1,90.2,-0.1,NaN,0.0,5.0,7.0,...,37.0,41.0,0,4,6,26,31,34,586.0,4.843
7,NaN,in fachlicher Hinsicht,112.0,71.4,74.0,-2.6,NaN,3.0,4.0,25.0,...,32.0,19.0,3,4,22,26,29,17,476.0,4.250
8,Vorbereitung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,in persönlichkeitsbildender Hinsicht,112.0,72.3,74.9,-2.6,NaN,7.0,7.0,17.0,...,35.0,10.0,6,6,15,32,31,9,451.0,4.027
10,Studium (5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,insgesamt,112.0,81.3,80.5,+0.8,NaN,3.0,3.0,15.0,...,37.0,14.0,3,3,13,36,33,13,483.0,4.312


### Nächster Komplettanlauf
Jetzt, da dies geklärt ist...

In [38]:

# Das hier sind Dinge, die man von Anfang an festlegt (das Jahr manuell)
jahr = 2006
dic_list = []


for table in range(0, len(umfrage), 2):
    df_schule = umfrage[table].df
    
    if df_schule.loc[0,13] == "Beurteilung":   #Wir checken hier anhand eines eindeutigen Merkmals, ob eine Zeile fehlt
        df_newrow = df_schule.head(1)
        df_schule = pd.concat([df_newrow, df_schule], ignore_index = True)   # Ignore Index ist entscheidend!

    name = df_schule.loc[2,0]
        
    if df_schule.loc[1,12] == "Beurteilung": # wiederholen den CHeck wegen Spaltenverschiebungen
        df_kurz = df_schule.drop([0,1,2,3,4,5,81,82])  # auch Schluss kürzen, weils da Müll drinhat, der stört
        df_kurz = df_kurz.replace("", np.nan)
        df_kurz.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19] # Kolumnenamen umschreiben
    
    else:
        df_kurz = df_schule.drop([0,1,2,3,4,5])
        df_kurz = df_kurz.replace("", np.nan)
    
    our_columns = [3,8,9,10,11,12,13]
    df_kurz[our_columns] = df_kurz[our_columns].astype(float)
    df_kurz["Summe"] = df_kurz[8] + df_kurz[9].multiply(2) + df_kurz[10].multiply(3) + df_kurz[11].multiply(4) + df_kurz[12].multiply(5) + df_kurz[13].multiply(6)
    df_kurz["Notenschnitt"] = (df_kurz["Summe"] / df_kurz[3]).round(3) #gerundet auf eine Kommastelle
    
    zufr = df_kurz.loc[6, "Notenschnitt"]
    vorb_f = df_kurz.loc[7, "Notenschnitt"]
    vorb_p = df_kurz.loc[9, "Notenschnitt"]
    prob = df_kurz.loc[13, "Notenschnitt"]
    urt = df_kurz.loc[14, "Notenschnitt"]
    zus = df_kurz.loc[16, "Notenschnitt"]
    konf = df_kurz.loc[17, "Notenschnitt"]

    mini_dict = {"jahr": jahr, 
                 "name": name, 
                 "zufr": zufr, 
                 "vorb_f": vorb_f, 
                 "vorb_p":vorb_p, 
                 "prob": prob, 
                 "urt": urt, 
                 "zus": zus, 
                 "konf": konf}
    dic_list.append(mini_dict)

In [46]:
df_final = pd.DataFrame(dic_list)

In [47]:
df_final

,jahr,name,zufr,vorb_f,vorb_p,prob,urt,zus,konf
0,2006,Büelrain,4.815,4.177,4.065,4.277,4.200,4.154,3.738
1,2006,Enge,4.746,4.074,3.926,4.493,4.437,3.986,3.761
2,2006,Freudenberg,4.316,4.000,3.868,4.526,4.658,4.079,3.842
3,2006,Glattal,4.833,3.941,4.118,4.500,4.333,3.722,3.444
4,2006,Hohe Promenade,5.091,4.758,4.636,4.706,4.706,4.000,4.000
5,2006,Hottingen,4.051,3.500,3.711,3.974,4.000,3.821,3.513
6,2006,Im Lee,4.676,4.406,3.971,4.603,4.473,3.836,3.493
7,2006,KME,5.037,4.173,4.257,4.309,4.185,3.963,3.407
8,2006,Küsnacht,5.095,4.119,4.524,4.738,5.095,4.000,4.048
9,2006,Liceo Artistico,5.450,4.579,4.895,4.950,4.950,4.950,4.263


# Heureka! Jetzt stimmts endlch!
Also schnell noch exportieren, das Zeugs!

In [49]:
df_final.to_csv("Pandas_Results2006.csv")